In [1]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import time
import numpy as np
from sklearn.model_selection import KFold
import statistics

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import f1_score
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/adenweiser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
def data_preprocessing(path, test=False):
    #start_time = time.time()
    
    #create appropriate file path
    if test == False:
        pfilename = path + "/product_training.json"
        rfilename = path + "/review_training.json"
    else:
        pfilename = path + "/product_test.json"
        rfilename = path + "/review_test.json"
    
    #extract files as pandas dataframes
    product_df = pd.read_json(pfilename)
    
    review_df = pd.read_json(rfilename).drop_duplicates(subset=["reviewerID", "unixReviewTime"], keep="first")
    ## 11.66 seconds to get to here
    
    review_df.drop(columns=["reviewerID","vote", "unixReviewTime","reviewTime","style","reviewerName","image"], axis=1 ,inplace=True)
    
    review_df['reviewText'].fillna("", inplace=True)
    review_df['summary'].fillna("", inplace=True)
    
    review_df.sort_values('asin', inplace = True)
    product_df.sort_values('asin', inplace = True)
    
    group = review_df.groupby("asin")
    
    #review_group_df = pd.DataFrame(columns = ['asin', 'numReviews', 'percentVerified','reviewText','summaryText', 'awesomeness'])
    
    # about the same amount of time to get to here
    start_time = time.time()
    datalist = []
    count = 0
    #awesome_pos = 0
    for asin, data in group:
        verifiedCount = data['verified'].sum()
        reviewCount = data['asin'].count()
        percentVerified = verifiedCount / reviewCount
        if count == 0:
            print(type(data['reviewText']))
        reviewText = ' '.join(data['reviewText'])
        #reviewText = ' '.join(transform_document(x) for x in data['reviewText'])
        #summaryText = ""
        summaryText = ' '.join(data['summary'])
        #summaryText = ' '.join(transform_document(x) for x in data['summary'])
        #reviewText = transform_document(' '.join(data['reviewText']))
        #summaryText = transform_document(' '.join(data['summary']))
        #awesomeness = 0
        
        #SENTIMENT ANALYSIS CHUNK
        (rev_mean, rev_stdev) = sentiment_analysis(data['reviewText'])
        (sum_mean, sum_stdev) = sentiment_analysis(data['summary'])
        while (product_df['asin'][count] != asin):
               count = count + 1
        
        awesomeness = product_df['awesomeness'][count]
        #awesome_pos = awesome_pos + reviewCount
        #awesomeness = product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0] #might be slow
        datalist.append([asin,  reviewCount, percentVerified, reviewText, summaryText, rev_mean, rev_stdev, sum_mean, sum_stdev, awesomeness])
        
        count = count + 1
        #if count > 100:
        #    break
        
        '''new_row = {'asin': asin, 
                   'numReviews': reviewCount, 
                   'percentVerified': percentVerified, 
                   'reviewText': transform_document(' '.join(data['reviewText'])), 
                   'summaryText': transform_document(' '.join(data['summary'])), 
                   'awesomeness': product_df.loc[product_df['asin'] == asin, 'awesomeness'].values[0]} 
        review_group_df = review_group_df.append(new_row, ignore_index = True)
         '''
    review_group_df = pd.DataFrame(datalist,columns =['asin', 'numReviews', 'percentVerified','reviewText','summaryText', \
                                                      'reviewMean', 'reviewStDev', 'summaryMean', 'summaryStDev', 'awesomeness'])    
    
    review_group_df.to_json(r'../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data.json')
    end_time = time.time()
    print(end_time - start_time)
    
    return review_group_df
    

In [5]:
# Preprocessed Data Generated (Reviews and Summaries aggregated, no NLP processing)
review_group_df = data_preprocessing("../devided_dataset_v2/CDs_and_Vinyl/train")
#review_group_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data.json')
#review_group_df
#review_group_df.to_json("preprocessed.json")
#review_group_df.head()
#review_group_df

,asin,numReviews,percentVerified,reviewText,summaryText,reviewMean,reviewStDev,summaryMean,summaryStDev,awesomeness
0,0000B049F5B33CD310EB1AB236E20191,3,0.666667,"Even tho I love this album, I am having proble...","I LOVE NANCY THE BEAUTIFUL LEGEND, NANCY WILSO...",1.969500,0.025729,1.487900,0.433071,1
1,00018184A9EC4D270219A296B2580303,14,0.071429,I have been a fan of GU's releases since i can...,One of GU's best The King of Progressive House...,1.710100,0.477970,1.208650,0.245517,0
2,000281A9CAC43FF1F335726A390636DA,1,0.000000,I made the mistake buying this album after lis...,Bad Business,1.285600,0.000000,0.457700,0.000000,0
3,00030884DF109F325638A6BFD5B13CFF,20,0.550000,Wow! A must hear! Bob Marley at his best. Wha...,Maybe the Greatest Live Reggae Album Ever Bob ...,1.708190,0.440495,1.278090,0.336915,1
4,000325BA25966B5FC701D5D2B5DBA4E0,3,1.000000,"Soft, melodic notes dot moving waves of ethere...",Light Notes Robin Miller/Transcendence relaxin...,1.895167,0.106837,0.872000,0.221703,0
...,...,...,...,...,...,...,...,...,...,...
71538,FFFDD3C72D23AF858D6E0ED92612370D,41,0.341463,The bright yellow case on Kiss' sophomore albu...,"'Cause baby's got the feeling, baby wants a sh...",1.774800,0.354987,1.227337,0.395480,1
71539,FFFDDE284A73B29B320381487EC7DE9E,4,0.500000,I picked up this CD for about $6 US and I have...,A leisurely stroll through the country AN HONE...,1.949700,0.049274,1.168350,0.385339,1
71540,FFFEB3EE2372807964F024707D50FB21,2,1.000000,"Pop music has a short memory, but Kevin Roland...",A strong comeback by a troubled artist The 4th...,1.955100,0.051902,1.038600,0.054589,0
71541,FFFF4545AB232D81D0F9B208388BB7AA,5,0.600000,"I ordered the album as soon as I stumbled it, ...","EXCELLENT concept album, but shouldn't be the ...",1.783980,0.331717,1.393140,0.293603,1


In [6]:
def sentiment_analysis(docs):
    sentiments = []
    sid = SentimentIntensityAnalyzer()
    for doc in docs:
        polarities = sid.polarity_scores(doc)
        sentiments.append(polarities['compound'])
    if len(sentiments) == 1:
        return (sentiments[0], 0)
    else:
        return (statistics.mean(sentiments) + 1, statistics.stdev(sentiments))

In [57]:
string_transformer = Pipeline(
    steps = [('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer())]
)
wordbagger = ColumnTransformer(
    transformers=[("rev", string_transformer, 'reviewText'), 
                  ("sum", string_transformer, 'summaryText')]
    , remainder='passthrough'
)

clf = Pipeline(steps = [("wordbag", wordbagger), 
                        ("scale", MaxAbsScaler()), 
                        ('classifier', RandomForestClassifier())])
#x = review_group_df.filter(['numReviews', 'percentVerified', 'reviewText', 'summaryText'])
#clf = LogisticRegression(tol = 0.001, max_iter = 150)
#clf.fit(x,y)
review_features = review_group_df.filter(['numReviews', 'percentVerified', 'reviewText', 'summaryText', 'reviewMean', 'reviewStDev', 'summaryMean', 'summaryStDev'])
y = review_group_df.filter(['awesomeness'])

In [58]:
start = time.time()
print("I started at " + str(start))
# this runs the k-fold cross-validation automatically?
cv10_results = cross_validate(clf, review_features, np.ravel(y), cv=10, n_jobs = -1, scoring = ['f1_macro', 'precision', 'recall'])
end = time.time()
print((end - start)/60)

I started at 1682714630.908139
39.14513936837514


In [59]:
print(cv10_results)
print(statistics.mean(cv10_results['test_f1_macro']))
print(statistics.stdev(cv10_results['test_f1_macro']))
print(statistics.mean(cv10_results['test_precision']))
print(statistics.stdev(cv10_results['test_precision']))
print(statistics.mean(cv10_results['test_recall']))
print(statistics.stdev(cv10_results['test_recall']))

{'fit_time': array([1573.13251805, 1579.55282402, 1579.21438384, 1597.68321085,
       1583.75677896, 1578.28207994, 1598.66020703, 1593.00883818,
        740.78515697,  739.19833899]), 'score_time': array([15.5390501 , 14.13390803, 13.83592224,  9.6982522 , 14.11366105,
       14.00288701,  8.36288905,  9.00348902,  6.05653405,  6.30660605]), 'test_f1_macro': array([0.57882855, 0.57741459, 0.5854562 , 0.58193651, 0.58244206,
       0.58275959, 0.57121476, 0.58086328, 0.58859251, 0.57945204]), 'test_precision': array([0.59479475, 0.59380306, 0.59961929, 0.5973831 , 0.59725115,
       0.59749739, 0.58921933, 0.596255  , 0.60203649, 0.59514768]), 'test_recall': array([0.73993156, 0.73650961, 0.74624901, 0.73308765, 0.75493551,
       0.75414583, 0.7509871 , 0.74598579, 0.74703869, 0.74275935])}
0.5808960096461864
0.004704425492159298
0.5963007227741477
0.003454509632326464
0.7451630102486714
0.00722133099383289


In [30]:
solvers = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
jobs = [-1]
maxiter = [100, 500, 1000]
c = [10, 1, 0.1]
param_grid={'classifier__C':c, 'classifier__solver':solvers, 'classifier__n_jobs':jobs, 'classifier__max_iter':maxiter}
grid_clf = GridSearchCV(clf, param_grid, n_jobs=-1)
start = time.time()
print("I'm starting!")
grid_clf.fit(review_features, np.ravel(y))
end = time.time()
print((end - start)/60)

I'm starting!


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


168.0281353791555


In [60]:
review_group_df.to_json(r'../devided_dataset_v2/CDs_and_Vinyl/train/cleaned_data.json')

In [38]:
pd.DataFrame.from_dict(grid_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__max_iter,param_classifier__n_jobs,param_classifier__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,175.314274,2.773403,22.106461,0.789819,10,100,-1,liblinear,"{'classifier__C': 10, 'classifier__max_iter': ...",0.575093,0.573276,0.584527,0.579676,0.584987,0.579512,0.004766,43
1,215.385077,9.125162,20.650041,0.633041,10,100,-1,newton-cg,"{'classifier__C': 10, 'classifier__max_iter': ...",0.575162,0.573206,0.584457,0.579746,0.585057,0.579526,0.004774,40
2,127.930750,3.097294,23.014140,0.660145,10,100,-1,lbfgs,"{'classifier__C': 10, 'classifier__max_iter': ...",0.576770,0.575302,0.584108,0.583240,0.584428,0.580770,0.003912,36
3,180.960551,4.390096,22.591533,0.792428,10,100,-1,sag,"{'classifier__C': 10, 'classifier__max_iter': ...",0.585296,0.589489,0.594381,0.596310,0.595401,0.592175,0.004169,32
4,190.735928,6.145154,22.109327,1.555230,10,100,-1,saga,"{'classifier__C': 10, 'classifier__max_iter': ...",0.592564,0.598504,0.603536,0.603928,0.603998,0.600506,0.004476,18
5,148.245815,5.060626,23.119408,0.548793,10,500,-1,liblinear,"{'classifier__C': 10, 'classifier__max_iter': ...",0.575093,0.573276,0.584527,0.579676,0.584987,0.579512,0.004766,43
6,209.577232,9.761099,23.340993,0.487204,10,500,-1,newton-cg,"{'classifier__C': 10, 'classifier__max_iter': ...",0.575162,0.573206,0.584457,0.579746,0.585057,0.579526,0.004774,40
7,277.310260,1.043650,23.895615,0.333598,10,500,-1,lbfgs,"{'classifier__C': 10, 'classifier__max_iter': ...",0.575232,0.573345,0.584597,0.579676,0.585057,0.579582,0.004753,38
8,684.006727,4.841569,26.239602,1.332473,10,500,-1,sag,"{'classifier__C': 10, 'classifier__max_iter': ...",0.575232,0.578447,0.587393,0.582821,0.587294,0.582237,0.004815,35
9,683.569998,55.728508,24.401188,1.039373,10,500,-1,saga,"{'classifier__C': 10, 'classifier__max_iter': ...",0.577049,0.581243,0.588161,0.586455,0.590858,0.584753,0.004969,33


All the older stuff is below here:

In [108]:
kf = KFold(n_splits = 10, shuffle = True)
for i, (train_index, test_index) in enumerate(kf.split(review_features)):
    start = time.time()
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    x_train = review_features.loc[train_index, :]
    x_test = review_features.loc[test_index, :]
    y_train = np.ravel(y.loc[train_index, :])
    y_test = np.ravel(y.loc[test_index,:])
    
    #x_train_features = wordbagger.fit_transform(x_train)
    #x_test_features = wordbagger.fit_transform(x_test)
    #scaler = preprocessing.MaxAbsScaler().fit(x_train_features)
    #x_train_scaled = scaler.transform(x_train_features)
    #print(type(y_train))
    #print(x_train.shape)
    #print(y_train.shape)
    #clf = Pipeline(steps = [("preprocess", preprocessor), ('classifier', LogisticRegression()) ])
    clf.fit(x_train, y_train)
    #print(x_train)
    #print(y_train.shape)
    #X_trans = preprocessor.fit_transform(x_train)
    #print(x_train.shape)
    print("model score: %.3f" % clf.score(x_test, y_test))
    end = time.time()
    print(end - start)

Fold 0:
  Train: index=[    0     1     2 ... 71540 71541 71542]
  Test:  index=[    7    10    21 ... 71512 71515 71520]
model score: 0.569
46.16732168197632
Fold 1:
  Train: index=[    0     1     2 ... 71540 71541 71542]
  Test:  index=[    3     4    11 ... 71526 71530 71538]
model score: 0.579
43.28184628486633
Fold 2:
  Train: index=[    0     1     2 ... 71540 71541 71542]
  Test:  index=[    8    16    30 ... 71504 71525 71527]
model score: 0.578
48.57866644859314
Fold 3:
  Train: index=[    0     1     2 ... 71539 71540 71542]
  Test:  index=[   25    27    44 ... 71487 71505 71541]
model score: 0.579
43.54520773887634
Fold 4:
  Train: index=[    1     2     3 ... 71540 71541 71542]
  Test:  index=[    0    12    15 ... 71536 71537 71539]
model score: 0.568
48.14018964767456
Fold 5:
  Train: index=[    0     1     2 ... 71539 71540 71541]
  Test:  index=[   17    19    22 ... 71531 71535 71542]
model score: 0.586
50.04946684837341
Fold 6:
  Train: index=[    0     1     2 ... 

NLP Functions

In [7]:
corpus = ["I am being handed a list of documents", "Each of these documents has several unique words", "The words will represent the class of each review", "I am also removing stopwords in order to make this make more sense"]
cleaned_corpus = [transform_document(doc) for doc in corpus]
vocabulary = vocabulary_from_corpus(cleaned_corpus, True)
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)), 
                 ('tfid', TfidfTransformer())]).fit(cleaned_corpus)),

In [12]:
def get_stopwords():
    file = open('en.txt')
    stopwords = []
    for line in file:
        stopwords.append(line.rstrip())
    return stopwords

In [23]:
def transform_corpus(review_group):
    stopwords = get_stopwords()
    reviewTextSet = review_group['reviewText']
    for index in review_group.index:
        curr_parsed = nlp(reviewTextSet[index].lower())
        doclist = []
        for token in curr_parsed:
            lemma = token.lemma_
            if not(re.match("[a-z0-9]+", lemma)):
                continue
            if lemma not in stopwords:
                doclist.append(lemma)
        reviewTextSet[index] = " ".join(doclist)

In [197]:
def transform_document(doc, remove_stopwords = True):
    #new_doc = ""
    stopwords = get_stopwords() # is this slow?
    parsed_text = nlp(doc) # is this slow
    doclist = []
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            doclist.append(lemma) # this is less slow?
    return " ".join(doclist)

In [50]:
def bag_of_words(review_text, remove_stopwords = True):
    word_bag = {}
    stopwords = get_stopwords()
    parsed_text = nlp(review_text)
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            if lemma in word_bag:
                word_bag[lemma] += 1
            else:
                word_bag[lemma] = 1
    return word_bag

In [51]:
def vocabulary_from_corpus(corpus, remove_stopwords = True):
    vocab_set = set()
    for document in corpus:
        word_bag = bag_of_words(document, remove_stopwords)
        for word in word_bag.keys():
            vocab_set.add(word)
    return list(vocab_set)
vocabulary_from_corpus(['this is the first document', 'this document is the second document', 'and this is the third one', 'is this the first document'], False)

['first', 'the', 'second', 'third', 'be', 'and', 'this', 'one', 'document']

In [88]:
product_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/product_training.json')
review_df = pd.read_json('../devided_dataset_v2/CDs_and_Vinyl/train/review_training.json')
#len(product_df.index)

In [6]:
teststring = "First, you need to preprocess the raw text data. This may involve tasks like tokenizing the text (i.e., splitting it into individual words), removing stopwords, stemming or lemmatizing the words, and converting the text into a numerical format that can be used as input for the model. Then, you need to split the data into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate its performance."
#teststring.lower()
parsed = nlp(teststring.lower())